In [1]:
from SStack import SStack
import cpca
from cpca import drawer
from DateFormatconvert2 import DateFormatHelper2
from cilin import CilinSimilarity
from queue import Queue,LifoQueue
import collections
import json
import re
import copy
import Levenshtein #字符串的编辑距离
import json

with open("./fang_baidu.json") as f:
     fang_baidu=json.load(f)
with open("./fang_360.json") as f:
     fang_360=json.load(f)
with open("./fang_hudong.json") as f:
     fang_hudong=json.load(f)
with open("./fang_sougou.json") as f:
     fang_sougou=json.load(f)
with open("./fang_wiki.json") as f:
     fang_wiki=json.load(f)

key_list = list(fang_baidu.keys())
cs = CilinSimilarity()
#用于将地址信息进行对齐统一
def merge_local(fang_360, fang_baidu):
    location = set()
    if fang_360=='' and fang_baidu=='':
        return fang_360
    if fang_360!='':
        location.add(fang_360)
    if fang_baidu!='':
        location.add(fang_baidu)
    if len(location)==1:
        return list(location)[0]
    df = cpca.transform(location) #实现地址信息格式的统一
    province = set()
    city = set()
    zone = set()
    address = set()
    for i in range(len(df['省'])):
        if df['省'][i]:
            province.add(df['省'][i])
        if df['市'][i]:
            city.add(df['市'][i])
        if df['区'][i]:
            zone.add(df['区'][i])
        if df['地址'][i]:
            address.add(df['地址'][i])
    if len(province)>1 or len(city)>1 or len(zone)>1 or len(address)>1: #表示地址描述不一致，需要消歧
        return 1
    else:
        last_location = ''
        if len(province)==1:
            last_location = last_location + list(province)[0]
        if len(city)==1:
            last_location = last_location + list(city)[0]
        if len(zone)==1:
            last_location = last_location + list(zone)[0]
        if len(address)==1:
            last_location = last_location + list(address)[0]
        return last_location
    
# 用于将日期信息进行对齐统一
def merge_date(fang_360, fang_baidu):
    date = set()
    if fang_360!='': #时间不为空的时候才放入集合进行比较
        formal_date = DateFormatHelper2.str2date(fang_360) #将日期格式进行统一
        date.add(formal_date)
    if fang_baidu!='':
        formal_date = DateFormatHelper2.str2date(fang_baidu)
        date.add(formal_date)
    if len(date)==1: #如果日期全部一样直接对齐
        date = list(date)[0]
        return date
    else: #否则判断年月日是否一样
        year = set()
        month = set()
        day = set()
        for j in list(date):
            if j.find('-')>0:
                res = j.split('-')
                if res[0]!='0':
                    year.add(res[0])
                if res[1]!='0':
                    month.add(res[1])
                if res[2]!='0':
                    day.add(res[2])
            else:
                year.add(j)
        if len(year)>1 or len(month)>1 or len(day)>1:
            return 1 #表示日期不一致，无法对齐，需要消歧
        else:
            last_date = ''
            if len(year)==1:
                last_date = last_date + list(year)[0]
            if len(month)==1:
                last_date = last_date + '-' + list(month)[0]
            if len(day)==1:
                last_date = last_date + '-' + list(day)[0]
            return last_date #返回对齐合并后的日期
        
# 用于将合并后的为单属性值列表的内容与另一个百科的内容合并（地点）
def merge_single_attribute_list(baike_1, baike_2, source, index):
    if baike_2=='':
        return baike_1
    for i in baike_1:
        for key,value in i.items():
            if (key=='起始时间' or key=='结束时间' or key=='出生日期') and merge_date(value, baike_2)!=1: #表示内容是一致的
                i['权重'] += 1
                i['来源'] = i['来源']+","+source
                return baike_1
            elif (key=='原籍' or key=='出生地') and merge_local(value, baike_2)!=1: #表示内容是一致的
                i['权重'] += 1
                i['来源'] = i['来源']+","+source
                return baike_1
            elif key!='消歧标记' and key!='来源' and key!='权重':
                if value==baike_2 or cs.similarity(value, baike_2)==1:
                    i['权重'] += 1
                    i['来源'] = i['来源']+","+source
                    return baike_1
            else:
                pass
    tmp_dict = dict()
    tmp_dict[index] = baike_2
    tmp_dict['消歧标记'] = 1
    tmp_dict['来源'] = source
    tmp_dict['权重'] = 1
    baike_1.append(tmp_dict)
    return baike_1

# 两个字符串的比较(看一个字符串是否是另一个字符串的子集)
def string_compile(str1, str2):
    str1 = re.split('_|\,|\/|\，|\、|\\|\-|\ ' ,str1)
    str2 = re.split('_|\,|\/|\，|\、|\\|\-|\ ' ,str2)
    str1_len = len(str1)
    str2_len = len(str2)
    var_set = set(str1)
    return_str = ''
    for i in str2:
        var_set.add(i)
    if len(var_set) == str1_len:
        j = 0 
        for i in str1:
            if j+1 == str1_len:
                return_str = return_str+i
            else:
                return_str = return_str+i+'/'
            j += 1
        return return_str
    elif len(var_set) == str2_len:
        j = 0 
        for i in str2:
            if j+1 == str2_len:
                return_str = return_str+i
            else:
                return_str = return_str+i+'/'
            j += 1
        return return_str
    else:
        return 1
    
# 5个文件都是列表时的对齐
def compile_list(list_360, list_baidu, source1, source2):
    if list_baidu=='':
        return list_360
    if list_360=='':
        return list_baidu
    year = set() #年份集合
    list_360_new = []
    list_baidu_new = []
    for dict_info1 in list_360: #依次取出360百科文件列表里的元素，对其中是时间的键值进行格式统一(只取年份)
        for key,value in dict_info1.items():
            if key=='时间点' or key=='起始时间':
                value = get_year(value)
                if value!='':
                    year.add(value) #将年份添加到集合中,对齐比较时，只看相同的起始时间的内容
                dict_info1[key] = value
            if key=='结束时间':
                value = get_year(value)
                dict_info1[key] = value
        list_360_new.append(dict_info1)
    for dict_info2 in list_baidu: #依次取出百度百科文件列表里的元素，对其中是时间的键值进行格式统一(只取年份)
        for key,value in dict_info2.items():
            if key=='时间点' or key=='起始时间':
                value = get_year(value)
                if value!='':
                    year.add(value)
                dict_info2[key] = value
            if key=='结束时间':
                value = get_year(value)
                dict_info2[key] = value
        list_baidu_new.append(dict_info2)
    return_list = [] #最终返回的字典集合
    flag2 = 0 #用于标记是否出现需要消歧的情况(在当前子属性下)
    for i in list(year):
        tmp_list1 = [] #中间过度列表，用于存放两个文件中年份相同的字典
        tmp_list2 = [] #中间过度列表，用于存放两个文件中年份相同的字典
        for dict_info1 in list_360_new:
            for key,value in dict_info1.items():
                if (key=='起始时间' or key=='时间点') and value == i:
                    tmp_list1.append(dict_info1)
        tmp_list1_copy = copy.deepcopy(tmp_list1) #因为下面在做的过程中会根据对齐消歧的情况不断改变tmp_list1的值，所以先用一个变量保存
        for dict_info2 in list_baidu_new:
            for key,value in dict_info2.items():
                if (key=='起始时间' or key=='时间点') and value == i:
                    tmp_list2.append(dict_info2)
        if len(tmp_list1)==0 and len(tmp_list2)!=0:
            for q in tmp_list2:
                return_list.append(q)
        elif len(tmp_list1)!=0 and len(tmp_list2)==0:
            for q in tmp_list1:
                return_list.append(q)
        else:
            keys = list(tmp_list1[0].keys())
            flag_list = [] #记录360百科文件与百度百科文件的对齐的字典部分
            for content_year_baidu in tmp_list2:
                flag = 0
                tmp_dict1 = copy.deepcopy(content_year_baidu) #用另一个中间变量来表示当前遍历到的字典
                for content_year_360 in tmp_list1:
                    flag5 = 0 #标识结束时间是否不相等
                    flag6 = 0 #标识其他非关键信息是否一样
                    flag7 = 0 #标识是否是时间点
                    flag4 = 0 #标记是否需要消歧
                    temp_dict = {} #用于存储两个文件对齐字典的中间结果
                    createVar = locals() #动态创建全局变量
                    tmp_dict2 = copy.deepcopy(content_year_360) #用另一个中间变量来表示当前遍历到的字典
                    for tmp_key in keys:
                        #这里之所以会只看这些键的值，是因为这些键的值更具有唯一性，先看具有唯一性的键值是否一样，一样才继续进行下面其他键值的比较，不一样则认为两个字典不是需要对齐的块
                        if tmp_key!='时间点' and tmp_key!='起始时间' and tmp_key!='所在单位' and tmp_key!='信息公布权威机关' and tmp_key!='组织机构' and tmp_key!='方向' and tmp_key!='颁奖单位' and tmp_key!='演讲/报告题目' and tmp_key!='事件' and tmp_key!='语录' and tmp_key!='争议起因' and tmp_key!='事件起因' and tmp_key!='教育理念/指导学生' and tmp_key!='专利名称' and tmp_key!='研究成果名称' and tmp_key!='项目名称' and tmp_key!='散文名称' and tmp_key!='论著名称' and tmp_key!='消歧标记' and tmp_key!='来源':
                            createVar['tmp_360_'+tmp_key] = content_year_360[tmp_key] #分别保存两个文件该键值对应的值
                            createVar['tmp_baidu_'+tmp_key] = content_year_baidu[tmp_key]
                        if tmp_key=='所在单位' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='信息公布权威机关' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                            if content_year_360[tmp_key] == content_year_baidu[tmp_key]:
                                flag = 1
                            elif cs.similarity(content_year_360[tmp_key], content_year_baidu[tmp_key])==1:
                                flag = 1
                            elif Levenshtein.distance(content_year_360[tmp_key],content_year_baidu[tmp_key])<4:
                                flag = 1
                            else:
                                continue
                    if flag==1:
                        specific_key = ['职称', '职位_职称', '所获奖项']
                        for tmp_key in keys:
                            if tmp_key=='时间点' or tmp_key=='起始时间':
                                temp_dict[tmp_key] = i
                                if tmp_key=='时间点':
                                    flag7 = 1
                            elif tmp_key=='所在单位' or tmp_key=='信息公布权威机关' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                                temp_dict[tmp_key] = content_year_360[tmp_key]
                            else:
                                if tmp_key=='结束时间':
                                    if eval('tmp_360_'+tmp_key)==eval('tmp_baidu_'+tmp_key):
                                        temp_dict[tmp_key] = content_year_360[tmp_key]
                                        for k in specific_key:
                                            if k in keys:
                                                if content_year_360[k] == content_year_baidu[k]:
                                                    temp_dict[k] = content_year_360[k]
                                                elif content_year_360[k]!='' and content_year_baidu[k]=='':
                                                    temp_dict[k] = content_year_360[k]
                                                elif content_year_360[k]=='' and content_year_baidu[k]!='':
                                                    temp_dict[k] = content_year_baidu[k]
                                                else:
                                                    str_res = string_compile(content_year_360[k], content_year_baidu[k])
                                                    if str_res!=1:
                                                        temp_dict[k] = str_res
                                                    else:
                                                        flag2 += 1
                                                        flag4 = 1
                                                        tmp_dict1['消歧标记'] = flag2
                                                        tmp_dict1['来源'] = source1
                                                        tmp_dict2['消歧标记'] = flag2
                                                        tmp_dict2['来源'] = source2
                                                        flag_list.append(content_year_360)
                                                        break #通过百科或其他方式实现对齐
                                        if flag4 ==1:
                                            break
                                    else:
                                        flag5 = 1
                                elif tmp_key=='地点':
                                    res = merge_local(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                    if res==1:
                                        flag2 += 1
                                        flag4 = 1
                                        tmp_dict1['消歧标记'] = flag2
                                        tmp_dict1['来源'] = source1
                                        tmp_dict2['消歧标记'] = flag2
                                        tmp_dict2['来源'] = source2
                                        flag_list.append(content_year_360)
                                        break #未对齐，需要消歧
                                    else:
                                        temp_dict[tmp_key] = res
                                else:
                                    if tmp_key!='消歧标记' and tmp_key!='来源':
                                        if eval('tmp_360_'+tmp_key) != eval('tmp_baidu_'+tmp_key):
                                            sim = cs.similarity(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                            str_res = string_compile(eval('tmp_360_'+tmp_key), eval('tmp_baidu_'+tmp_key))
                                            if sim == 1 or str_res!=1:
                                                if str_res!=1:
                                                    temp_dict[tmp_key] = str_res
                                                else:
                                                    temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key) #相似性得分1，即存在于词库中，直接返回
                                                if '结束时间' in keys:
                                                    if content_year_360['结束时间'] == content_year_baidu['结束时间']:
                                                        temp_dict['结束时间'] = get_year(content_year_360['结束时间'])
                                                    elif content_year_360['结束时间']!='' and content_year_baidu['结束时间']=='':
                                                        temp_dict['结束时间'] = get_year(content_year_360['结束时间'])
                                                    elif content_year_360['结束时间']=='' and content_year_baidu['结束时间']!='':
                                                        temp_dict['结束时间'] = get(content_year_baidu['结束时间'])
                                                    else:
                                                        flag2 += 1
                                                        flag4 = 1
                                                        tmp_dict1['消歧标记'] = flag2
                                                        tmp_dict1['来源'] = source1
                                                        tmp_dict2['消歧标记'] = flag2
                                                        tmp_dict2['来源'] = source2
                                                        flag_list.append(content_year_360)
                                                        break #通过百科或其他方式实现对齐
                                                else: #表示是时间点,且有除时间点和关键内容外的其他信息相一致，则认为对齐
                                                    flag_list.append(content_year_360)
                                                    temp_dict = content_year_baidu
                                                    if str_res!=1:
                                                        temp_dict[tmp_key] = str_res
                                                    else:
                                                        temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key)
                                                    break
                                            else:
                                                if '结束时间' in keys:
                                                    flag6 = 1
                                                else: #表示时间点情况下需要消歧
                                                    flag2 += 1
                                                    flag4 = 1
                                                    tmp_dict1['消歧标记'] = flag2
                                                    tmp_dict1['来源'] = source1
                                                    tmp_dict2['消歧标记'] = flag2
                                                    tmp_dict2['来源'] = source2
                                                    flag_list.append(content_year_360)
                                                    break
                                        else:
                                            temp_dict[tmp_key] = eval('tmp_baidu_'+tmp_key)
                                            if '结束时间' in keys:
                                                if content_year_360['结束时间'] == content_year_baidu['结束时间']:
                                                    temp_dict['结束时间'] = content_year_360['结束时间']
                                                elif content_year_360['结束时间']!='' and content_year_baidu['结束时间']=='':
                                                    temp_dict['结束时间'] = content_year_360['结束时间']
                                                elif content_year_360['结束时间']=='' and content_year_baidu['结束时间']!='':
                                                    temp_dict['结束时间'] = content_year_baidu['结束时间']
                                                else:
                                                    flag2 += 1
                                                    flag4 = 1
                                                    tmp_dict1['消歧标记'] = flag2
                                                    tmp_dict1['来源'] = source1
                                                    tmp_dict2['消歧标记'] = flag2
                                                    tmp_dict2['来源'] = source2
                                                    flag_list.append(content_year_360)
                                                    break #通过百科或其他方式实现对齐
                                            else: #表示时间点情况下是对齐的
                                                flag_list.append(content_year_360)
                                                temp_dict = content_year_baidu
                                                break
                                    else:
                                        pass
                        if flag5==1 and flag6==1: #结束时间和非关键其他信息都不相同时
                            pass
                        else:
                            flag_list.append(content_year_360)
                            break
                if flag4!=0: #先判断是否需要消歧，如果要消歧就直接把需要消歧的两个文件的内容加入返回列表
                    return_list.append(tmp_dict1)
                    return_list.append(tmp_dict2)
                    #tmp_list1.remove(content_year_360)
                else:
                    if flag==0:
                        return_list.append(content_year_baidu)
                    else:
                        if flag5==1 and flag6==1: #第一层循环的一行比较到第二个循环的最后一个时都没有实现对齐
                            return_list.append(content_year_baidu)
                        else:
                            return_list.append(temp_dict)
                            #tmp_list1.remove(content_year_360)
            for content_year_360 in tmp_list1_copy: #找出360百科文件中未与百度百科对齐的部分
                flag = 0
                for align_content_360 in flag_list:
                    for tmp_key in keys:
                        if tmp_key=='所在单位' or tmp_key=='信息公布权威机关' or tmp_key=='组织机构' or tmp_key=='方向' or tmp_key=='颁奖单位' or tmp_key=='演讲/报告题目' or tmp_key=='事件' or tmp_key=='语录' or tmp_key=='争议起因' or tmp_key=='事件起因' or tmp_key=='教育理念/指导学生' or tmp_key=='专利名称' or tmp_key=='研究成果名称' or tmp_key=='项目名称' or tmp_key=='散文名称' or tmp_key=='论著名称':
                            if content_year_360[tmp_key] == align_content_360[tmp_key]:
                                flag = 1 #做一个关键信息相等时的标记
                if flag==0:
                    return_list.append(content_year_360)
    return return_list

# 将结果写入到新json文件中
def list_to_newjson(word, res):
    if word=='本科' or word=='硕士研究生' or word=='博士研究生' or word=='博士后':
        align_json['人物履历']['教育经历'][word]=res
    elif word=='任职' or word=='任免_辞职':
        align_json['人物履历']['工作经历'][word]=res
    elif word=='学术兼职' or word=='技术工作兼职' or word=='社会兼职':
        align_json['其他职务'][word]=res
    elif word=='学术论著类' or word=='散文类' or word=='承担项目类' or word=='研究成果类' or word=='发明专利类' or word=='人才培养类':
        align_json["主要成果"][word]=res
    else:
        align_json[word]=res

# 只取日期前面的年份
def get_year(date):
    if date!='':
        date = date.strip()
        year = date[:4]
        year = year + '年'
        return year
    return date

# 维护一个键的队列
def preserve_key_queue(key, key_queue):
    if key=='任免_辞职':
        delete = key_queue.get()
        delete = key_queue.get()
    elif key=='社会兼职' or key=='人才培养类' or key=='博士后':
        delete = key_queue.get()
    else:
        pass

# 对非列表且是字典形式的内容进行对齐(这里主要关注基本信息、本科、硕士、博士、博士后这几个模块)
def align_dict(content_baike_1, content_baike_2, source1, source2):
    keys = list(content_baike_1.keys())
    new_dict = {} #用于存放对齐与未对齐(需消歧)后的内容
    for index in keys:
        tmp_dict1 = dict() #中间过度字典，用于存放两个文件未对齐的键与值
        tmp_dict2 = dict()
        tmp_dict3 = dict() #表示记录对齐时的权重
        tmp_dict1[index] = content_baike_1[index]
        tmp_dict2[index] = content_baike_2[index]
        flag = 0 #用于标记是否有未对齐的内容
        if index=='起始时间' or index=='结束时间' or index=='出生日期':
            if type(content_baike_2[index]).__name__ == 'list':
                new_dict[index] = merge_single_attribute_list(content_baike_2[index],content_baike_1[index],source2,index)
            elif type(content_baike_2[index]).__name__ != 'dict':
                res = merge_date(content_baike_1[index], content_baike_2[index])
                if res!=1:
                    tmp_dict3[index] = res
                    tmp_dict3['权重'] = 1
                    new_dict[index] = tmp_dict3
                else:
                    tmp_dict1[index] = get_year(tmp_dict1[index])
                    tmp_dict2[index] = get_year(tmp_dict2[index])
                    tmp_dict1['消歧标记'] = 1
                    tmp_dict1['来源'] = source1
                    tmp_dict1['权重'] = 1
                    tmp_dict2['消歧标记'] = 1
                    tmp_dict2['来源'] = source2
                    tmp_dict2['权重'] = 1
                    new_dict[index] = [tmp_dict1, tmp_dict2]
            else:
                res = merge_date(content_baike_1[index], content_baike_2[index][index])
                if res!=1:
                    tmp_dict3[index] = res
                    tmp_dict3['权重'] = content_baike_2[index]['权重']+1
                    new_dict[index] = tmp_dict3
                else:
                    tmp_dict1[index] = get_year(tmp_dict1[index])
                    tmp_dict1['消歧标记'] = 1
                    tmp_dict1['来源'] = source1
                    tmp_dict1['权重'] = 1
                    new_dict[index] = [tmp_dict1, content_baike_2[index]]
        elif index=='出生地' or index=='原籍':
            if type(content_baike_2[index]).__name__ == 'list':
                new_dict[index] = merge_single_attribute_list(content_baike_2[index],content_baike_1[index],source2,index)
            elif type(content_baike_2[index]).__name__ != 'dict':
                res = merge_local(content_baike_1[index], content_baike_2[index])
                if res!=1:
                    tmp_dict3[index] = res
                    tmp_dict3['权重'] = 1
                    new_dict[index] = tmp_dict3
                else:
                    tmp_dict1['消歧标记'] = 1
                    tmp_dict1['来源'] = source1
                    tmp_dict1['权重'] = 1
                    tmp_dict2['消歧标记'] = 1
                    tmp_dict2['来源'] = source2
                    tmp_dict2['权重'] = 1
                    new_dict[index] = [tmp_dict1, tmp_dict2]
            else:
                res = merge_local(content_baike_1[index], content_baike_2[index][index])
                if res!=1:
                    tmp_dict3[index] = res
                    tmp_dict3['权重'] = content_baike_2[index]['权重']+1
                    new_dict[index] = tmp_dict3
                else:
                    tmp_dict1['消歧标记'] = 1
                    tmp_dict1['来源'] = source1
                    tmp_dict1['权重'] = 1
                    new_dict[index] = [tmp_dict1, content_baike_2[index]]
        else:
            if type(content_baike_2[index]).__name__ == 'list':
                new_dict[index] = merge_single_attribute_list(content_baike_2[index],content_baike_1[index],source2,index)
            elif type(content_baike_2[index]).__name__ != 'dict':
                if content_baike_1[index] == content_baike_2[index]:
                    new_dict[index] = content_baike_2[index]
                elif content_baike_1[index]=='' and content_baike_2[index]!='':
                    new_dict[index] = content_baike_2[index]
                elif content_baike_1[index]!='' and content_baike_2[index]=='':
                    new_dict[index] = content_baike_1[index]
                else:
                    sim = cs.similarity(content_baike_1[index], content_baike_2[index])
                    if sim == 1:
                        new_dict[index] = content_baike_1[index] #相似性得分1，即存在于词库中
                    else:
                        tmp_dict1['消歧标记'] = 1
                        tmp_dict1['来源'] = source1
                        tmp_dict1['权重'] = 1
                        tmp_dict2['消歧标记'] = 1
                        tmp_dict2['来源'] = source2
                        tmp_dict2['权重'] = 1
                        new_dict[index] = [tmp_dict1, tmp_dict2]
            else:
                if content_baike_1[index] == content_baike_2[index][index]:
                    content_baike_2[index]['权重'] += 1
                    new_dict[index] = content_baike_2[index]
                elif content_baike_1[index]=='' and content_baike_2[index][index]!='':
                    new_dict[index] = content_baike_2[index]
                else:
                    sim = cs.similarity(content_baike_1[index], content_baike_2[index][index])
                    if sim == 1: #相似性得分1，即存在于词库中
                        content_baike_2[index]['权重'] += 1
                        new_dict[index] = content_baike_2[index]
                    else:
                        tmp_dict1['消歧标记'] = 1
                        tmp_dict1['来源'] = source1
                        tmp_dict1['权重'] = 1
                        new_dict[index] = [tmp_dict1, content_baike_2[index]]
    return new_dict
                                                        
# 对非列表非字典形式的内容对齐(也包括都是列表，但列表里面都是字符串，不是字典的情况)
def string_align(word, content_360, content_baidu):
    new_list = []
    baidu_align = [] #保存百度与360对齐部分
    if word == '人物影响' or word == '研究领域': #要么为空，要么字符串值在List里
        if content_360!='' and content_baidu=='':
            new_list = content_360
        elif content_360=='' and content_baidu!='':
            new_list = content_baidu
        elif content_360!='' and content_baidu!='':
            for i in content_360:
                flag = 0
                for j in content_baidu:
                    if i==j:
                        new_list.append(i)
                        baidu_align.append(j)
                        flag=1
                        break
                    else:
                        cs = CilinSimilarity()
                        sim = cs.similarity(i, j)
                        if sim == 1:
                            new_list.append(i)
                            baidu_align.append(j)
                            flag=1
                            break
                if flag==0:
                    new_list.append(i)
            for i in content_baidu:
                flag=0
                for j in baidu_align:
                    if i==j:
                        flag=1
                if flag==0:
                    new_list.append(i)
        else:
            new_list = ''
    else: #既然content_360能够传到这里来进行计算，说明content_360此时既不是列表也不是字典，所以只能为空(如果有值的话，值的形式要么为字典，要么为列表)
        if content_baidu=='':
            new_list = content_baidu
        else:
            new_list = content_baidu
            #当有多个文件的时候，可能存在多个文件都是列表的情况，此时就又到了列表的比较函数模块了
    return new_list

# 其他信息对齐
def other_align(st, key_queue, source1, source2):
    if not st.is_empty():
        content = set()
        word = st.pop()
        key_queue.put(word) #键值队列，用于保存前面已出现的键值(目的是方便后面迭代时能准确找到进行到什么地方了)
        baike_name_1 = ['360百科','百度百科']
        baike_name_2 = ['互动百科','搜狗百科','维基百科']
        if (source1 not in baike_name_1) and (source2 not in baike_name_1):
            if source1=='互动百科':
                key_str = 'fang_hudong'
            elif source1=='搜狗百科':
                key_str = 'fang_sougou'
            else:
                key_str = 'fang_wiki'
            for i in key_queue.queue: #通过维护的键的队列取到对应的键值
                key_str = key_str+'[\''+i+'\']'
            content_baike_1 = eval(key_str) #这里可能存在里面的内容仍然是list或dict的情况
            with open("./fang_merge.json") as f:
                 fang_merge=json.load(f)
            key_str = 'fang_merge'
            for i in key_queue.queue:
                key_str = key_str+'[\''+i+'\']'
            content_baike_2 = eval(key_str) #baike_2是合并后的内容
        else:
            key_str = 'fang_360'
            for i in key_queue.queue: #通过维护的键的队列取到对应的键值
                key_str = key_str+'[\''+i+'\']'
            content_baike_1 = eval(key_str) #这里可能存在里面的内容仍然是list或dict的情况
            key_str = 'fang_baidu'
            for i in key_queue.queue:
                key_str = key_str+'[\''+i+'\']'
            content_baike_2 = eval(key_str)
        if type(content_baike_1).__name__ == 'list': #如果是列表，则取出列表里的每一个字典
            if word=='人物影响' or word=='研究领域':
                res = string_align(word, content_baike_1, content_baike_2)
                list_to_newjson(word, res)
                key = key_queue.get()
                preserve_key_queue(key, key_queue)
                other_align(st, key_queue, source1, source2)
            else:
                #print()
                res = compile_list(content_baike_1, content_baike_2, source1, source2) #返回的是一个列表，列表里面是多个字典
                list_to_newjson(word,res)
                key = key_queue.get()
                preserve_key_queue(key, key_queue) #得到动态维护的键的队列
                other_align(st, key_queue, source1, source2)
        elif type(content_baike_1).__name__ == 'dict':
            if word=='基本信息' or word=='本科' or word=='硕士研究生' or word=='博士研究生' or word=='博士后':
                res = align_dict(content_baike_1, content_baike_2, source1, source2)
                list_to_newjson(word,res)
                key = key_queue.get()
                preserve_key_queue(key, key_queue)
                other_align(st, key_queue, source1, source2)
            else:
                dict_key1 = list(content_baike_1.keys()) #将字典里的键都放入栈中，稍后依次取出进行对齐
                dict_key1.reverse()
                for i in dict_key1:
                    st.push(i)
                other_align(st, key_queue, source1, source2)
        else:
            res = string_align(word, content_baike_1, content_baike_2)
            list_to_newjson(word, res)
            key = key_queue.get()
            preserve_key_queue(key, key_queue)
            other_align(st, key_queue, source1, source2)
    else:
        return 0

def align_entrace(source1, source2):
    tree=lambda:collections.defaultdict(tree)
    global align_json
    align_json=tree()
    st = SStack() #用栈存放每一轮的属性
    lq = LifoQueue(maxsize=0) #用后进先出队列用于存放文件操作过程中的中间键值(用于新建新的对齐json)
    key_list.reverse()
    for i in key_list:
        st.push(i)
    # source1 = '百度百科'
    # source2 = '360百科'
    res = other_align(st, lq, source1, source2)
    print("  ",json.dumps(align_json,ensure_ascii=False,indent=4))
    fang_merge = json.dumps(align_json,ensure_ascii=False,indent=4)
    with open('fang_merge.json', 'w') as f:
        f.write(fang_merge) #设置不转换成ascii  json字符串首缩进

if __name__ == '__main__':
    align_entrace('百度百科', '360百科')
    for i in ['搜狗百科','互动百科','维基百科']:
        align_entrace(i, '百科合并')

e:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\hly\AppData\Local\Temp\jieba.cache
Loading model cost 1.160 seconds.
Prefix dict has been built succesfully.
e:\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


   {
    "基本信息": {
        "中文名": "方滨兴",
        "外文名": "Bin xing Fang",
        "性别": "男",
        "出生日期": {
            "出生日期": "1960-7-17",
            "权重": 1
        },
        "出生地": {
            "出生地": "黑龙江省哈尔滨市",
            "权重": 1
        },
        "国籍": "中国",
        "民族": "汉族",
        "职业": [
            {
                "职业": "国家计算机网络应急技术处理协调中心主任",
                "消歧标记": 1,
                "来源": "百度百科",
                "权重": 1
            },
            {
                "职业": "科研工作者",
                "消歧标记": 1,
                "来源": "360百科",
                "权重": 1
            }
        ],
        "毕业院校": "哈尔滨工业大学",
        "政治面貌": "中国共产党",
        "代表作品": "《论网络空间主权》、《在线社交网络分析》",
        "主要成就": "2005年当选为中国工程院院士",
        "曾任职": "",
        "信仰": "无",
        "原籍": {
            "原籍": "江西省上饶市万年县",
            "权重": 1
        }
    },
    "人物履历": {
        "教育经历": {
            "本科": {
                "起始时间": {
                    "起始时间": "1978",
                    "

   {
    "获奖争议": "",
    "社会争议_争议事件": "",
    "个性寄语": "",
    "人物影响": "",
    "外界评价": "",
    "学术报告": {
        "时间点": "2014年11月19日",
        "演讲_报告题目": "《物联网搜索技术》",
        "组织单位_活动单位名称": "世界互联网大会",
        "地点": "",
        "重要观点": "",
        "全体观众": ""
    },
    "主要成果": {
        "学术论著类": [
            {
                "时间点": "2002年",
                "论著名称": "建设网络应急体系,保障网络空间安全"
            },
            {
                "时间点": "2017年",
                "论著名称": "《论网络空间主权》"
            },
            {
                "时间点": "1992年",
                "论著名称": "《计算机病毒及其对策》"
            },
            {
                "时间点": "1992年",
                "论著名称": "《计算机病毒及其对策》"
            },
            {
                "时间点": "2014年",
                "论著名称": "《在线社交网络分析》"
            },
            {
                "时间点": "1980年",
                "论著名称": "《ECLIPSE MV/8000超级小型机产品介绍》"
            },
            {
                "时间点": "2010年",
                "论著名称": "《网和天下 三网融合理论、实验与信息安全

   {
    "基本信息": {
        "中文名": "方滨兴",
        "外文名": "Bin xing Fang",
        "性别": "男",
        "出生日期": {
            "出生日期": "1960-7-17",
            "权重": 3
        },
        "出生地": {
            "出生地": "黑龙江省哈尔滨市",
            "权重": 3
        },
        "国籍": "中国",
        "民族": "汉族",
        "职业": [
            {
                "职业": "国家计算机网络应急技术处理协调中心主任",
                "消歧标记": 1,
                "来源": "百度百科,百科合并",
                "权重": 2
            },
            {
                "职业": "科研工作者",
                "消歧标记": 1,
                "来源": "360百科",
                "权重": 1
            },
            {
                "职业": "学者，教授",
                "消歧标记": 1,
                "来源": "百科合并",
                "权重": 1
            }
        ],
        "毕业院校": [
            {
                "毕业院校": "哈尔滨工业大学，清华大学",
                "消歧标记": 1,
                "来源": "互动百科",
                "权重": 1
            },
            {
                "毕业院校": "哈尔滨工业大学",
                "消歧标记

In [6]:
import os

def get_files(path='D:\zyt\\azyt\sfx', rule=".json"):
    all = []
    for fpathe,dirs,fs in os.walk(path):   # os.walk获取所有的目录
        for f in fs:
            filename = os.path.join(fpathe,f)
            if filename.endswith(rule):  # 判断是否是".sfx"结尾
                all.append(filename)
    return all

if __name__ == "__main__":
    b = get_files(r"./")
    for i in b:
        print("  ",i)

   ./fang_360.json
   ./fang_baidu.json
   ./fang_hudong.json
   ./fang_sougou.json
   ./fang_wiki.json
